# VNA V2

Clean up the code to simultaneously read from ADCs. Perform S-parameter caculations and graph output. Completes full data proccessing pipeline and outputs the S-Parameters 

In [5]:
import sys
import os
import importlib #
sys.path.append(os.path.abspath('../scripts'))

#widgets and display
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output
import pprint

#Plotting 
from bqplot import pyplot as plt
import threading
from threading import Thread

#caculations
import numpy as np 
from scipy import signal
import time
import math

# Import functions from custom scripts
import sig_source
importlib.reload(sig_source)
from sig_source import SigSource


In [6]:
#Import Pynq
from pynq import PL
from pynq import allocate
from pynq import Overlay
import xrfdc

# Upload Code to RFSOC

In [7]:
rfsoc_button = widgets.Button(description="Print RFSOC Code Lines")

out = Output()
def print_rfsoc(func):
    with out:
        out.clear_output
        try: 
            pprint.pprint(ol.ip_dict)
        except Exception as e:
            print(f"Error: {e}")


PL.reset() #important fixes caching issues which have popped up.
ol = Overlay('./design_1.bit')  #locate/point to the bit file
dma_interface = ol.axi_dma_0
rf = ol.usp_rf_data_converter_0

rfsoc_button.on_click(print_rfsoc)
display(widgets.VBox([widgets.Label(value="Print RFSOC Output"), rfsoc_button, out])) #Display button only after the code is uploaded 

# Signal Source Generation 

In [8]:
source = SigSource(start = 10000000, stop = 20000000000, resolution = 100)

start_stop_slider = widgets.FloatRangeSlider(
    value=[source.lowest_freq/(10**6), source.highest_freq/(10^6)],
    min= source.lowest_freq/(10**6),
    max= source.highest_freq/(10**6),
    step= 1,
    #description='Start Frequency (MHz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='1.0f',
    layout=widgets.Layout(width='500px') #Change layout width here!! 
)
center_slider = widgets.FloatSlider(value=source.center_freq/(10**6), min=source.lowest_freq/(10**6), max=source.highest_freq/(10**6), step=1)
span_slider = widgets.FloatSlider(value=((source.highest_freq - source.lowest_freq)/2)/10**6, min=1, max=((source.highest_freq - source.lowest_freq))/10**6, step=1)
resolution_slider = widgets.FloatSlider(value = 100, min = 0, max = 1000, step=1)
calc_selection = widgets.ToggleButtons(
    options=['Start-Stop', 'Center-Span'],
    description='Caculate Based On:',
    disabled=False,
    button_style=''
)
update_button = widgets.Button(description="Update Wave")

out = Output()
def update_function(change):
    with out:
        out.clear_output()
        try:
            if (calc_selection.value == "Start-Stop"):
                source.update_parameters(start = start_stop_slider.value[0]*10**6, 
                                        stop = start_stop_slider.value[1]*10**6, 
                                        resolution = resolution_slider.value)
            elif (calc_selection.value == "Center-Span"):
                source.update_parameters(center = center_slider.value*10**6,
                                        span = span_slider.value*10**6, 
                                        resolution = resolution_slider.value)
            
            start_stop_slider.value = (source.start/10**6, source.stop/10**6)
            center_slider.value = source.center/10**6
            span_slider.value = source.span/10**6
            #print("Start frequency: {} Stop Frequency: {}".format(start_stop_slider.value[0], start_stop_slider.value[1]) )
        except Exception as e:
            print(f"Error: {e}")


b1 = widgets.VBox([widgets.VBox([widgets.Label(value="Start Stop Slider (MHz)"), start_stop_slider]), 
                   widgets.HBox([widgets.VBox([widgets.Label(value="Center Slider (Mhz)"),center_slider]), 
                                 widgets.VBox([widgets.Label("Span Slider (Hz)"), span_slider])]),
                    widgets.VBox([widgets.Label(value="Points per Sweep"), resolution_slider]),
                    calc_selection,
                    update_button,
                    out],
                    layout=widgets.Layout(padding='20px 0'))

update_button.on_click(update_function)
display(b1)

# Real Time Output Plot

In [9]:
# Sampling frequency
fs = 147.456e6
# Number of samples
n = 65536
T = n/fs

def read_dma():
    # Trigger the DMA transfer and wait for the result
    out_buffer = allocate(400024 * 4, dtype=np.int32)
    # Trigger the DMA transfer and wait for the result
    dma_interface.recvchannel.transfer(out_buffer)
    dma_interface.recvchannel.wait()
    return out_buffer

def iq_break_data(in_data):
    val = in_data&0xFFFF
    if val >= 32768:
        real = np.int32(0xFFFF0000|val)
    else:
        real = val
    imag = in_data>>16
    return real, imag

# Function to process data
def iq_break_data_np(in_data):
    # Extract real part
    val = in_data & 0xFFFF
    real = np.where(val >= 32768, np.int32(0xFFFF0000 | val), val)
    
    # Extract imaginary part
    imag = in_data >> 16
    return real, imag

'''
Caculates the S-parameters of the two given signals
This caculation must be done to each of the 4 signals to find S11, S12, S21, S22
'''
def caculate_S_param (forward_signal, reverse_signal):
    # Convert signals to polar. This is done by a CORDIC on the FPGA.
    forward_magnitude = np.abs(forward_signal)
    forward_phase = np.angle(forward_signal)
    reverse_magnitude = np.abs(reverse_signal)
    reverse_phase = np.angle(reverse_signal)

    # Reflection coefficient magnitude comes from the ratios of the magnitudes of the forward and reverse signals.
    gamma_magnitude = np.mean(forward_magnitude) / np.mean(reverse_magnitude)

    # Reflection coefficient phase comes from the difference in phase of the forward and reverse signals. 
    # Unwrapping performed by subtracting 2pi when we are outside the 180 deg range. Not 100% sure this works under all conditions.
    phase_difference = forward_phase - reverse_phase
    phase_difference[phase_difference > math.pi] = phase_difference[phase_difference > math.pi] - 2 * math.pi
    gamma_phase = np.mean(phase_difference)

    return gamma_magnitude, gamma_phase

In [30]:
# Plot Size
PLOT_SIZE = 3e-6

# Initialize data lists
time_data = np.linspace(0, T, n)  # Time data (X-axis)
x_axis = [time_data, time_data, time_data, time_data]
# Initialize the plot
fig = plt.figure(title="Real-time Sensor Data", animation_duration=0)
line = plt.plot([], [], colors=["blue", "red", "green", "orange"])  # Initial empty plot

plt.xlim(0, PLOT_SIZE)  # Initial X-axis range, will update dynamically
plt.xlabel("Time [s]")

# Function to update the plot with new sensor data
def update_plot():
    while True:
        if is_running.value:
            out_buffer = read_dma()

            # Convert the entire out_buffer into a NumPy array
            out_buffer_np = np.array(out_buffer, dtype=np.int32)  # Assuming `out_buffer` is the raw input array

            # Split into 4 sub-arrays
            out_buffer0 = out_buffer_np[0::4]
            out_buffer1 = out_buffer_np[1::4]
            out_buffer2 = out_buffer_np[2::4]
            out_buffer3 = out_buffer_np[3::4]

            # Process all buffers at once
            real0, imag0 = iq_break_data_np(out_buffer0)
            real1, imag1 = iq_break_data_np(out_buffer1)
            real2, imag2 = iq_break_data_np(out_buffer2)
            real3, imag3 = iq_break_data_np(out_buffer3)
                
            #Caculate the S-parameters 
            #For caculating the S-parameters combine into complex lists
            array0 = real0 + 1j*imag0 
            array1 = real1 + 1j*imag1
            array2 = real2 + 1j*imag2 
            array3 = real3 + 1j*imag3 

            S11_mag, S11_phase = caculate_S_param(array0, array1)
            S12_mag, S12_phase = caculate_S_param(array0, array2)
            S21_mag, S21_phase = caculate_S_param(array1, array3)
            S22_mag, S22_phase = caculate_S_param(array2, array3)

#             print(f"Calculated S11 magnitude is {S11_mag:.2f}.")
#             print(f"Calculated S11 phase is {S11_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S12 magnitude is {S12_mag:.2f}.")
#             print(f"Calculated S12 phase is {S12_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S21 magnitude is {S21_mag:.2f}.")
#             print(f"Calculated S21 phase is {S21_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S22 magnitude is {S22_mag:.2f}.")
#             print(f"Calculated S22 phase is {S22_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}."
            
            # Update the plot with new data
            line.x = x_axis
            line.y = [real0, real1, real2, real3]
            
# Toggle button to start/stop the plot
is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(is_running, fig)

# Function to start the thread for continuous plotting
def start_plot(change):
    if is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        thread = Thread(target=update_plot, daemon=True)
        thread.start()

# Watch the button and start the plot when pressed
is_running.observe(start_plot, names='value')

ToggleButton(value=False, description='Running', icon='play', tooltip='Start/Stop the live plot')

Figure(axes=[Axis(label='Time [s]', scale=LinearScale(max=3e-06, min=0.0)), Axis(orientation='vertical', scale…

## Now Preparing for Sweeping Output

What needs to happen:
- Loop through setting each of the frequency points on the source
    - In the loop, set the frequency
    - Have some finite measurement in time
    - Read the DMA out for that time
    - Caculate the S-parameters
    - Save that into a plot

- The plot needs to live update linked to those results

- Use: source.generate_freq_points()

In [15]:
def extract_S_param(out_buffer):
    # Convert the entire out_buffer into a NumPy array
    out_buffer_np = np.array(out_buffer, dtype=np.int32)  # Assuming `out_buffer` is the raw input array

    # Split into 4 sub-arrays
    out_buffer0 = out_buffer_np[0::4]
    out_buffer1 = out_buffer_np[1::4]
    out_buffer2 = out_buffer_np[2::4]
    out_buffer3 = out_buffer_np[3::4]

    # Process all buffers at once
    real0, imag0 = iq_break_data_np(out_buffer0)
    real1, imag1 = iq_break_data_np(out_buffer1)
    real2, imag2 = iq_break_data_np(out_buffer2)
    real3, imag3 = iq_break_data_np(out_buffer3)

    #For caculating the S-parameters combine into complex lists
    array0 = real0 + 1j*imag0 
    array1 = real1 + 1j*imag1
    array2 = real2 + 1j*imag2 
    array3 = real3 + 1j*imag3 

    S11_mag, S11_phase = caculate_S_param(array0, array1)
    S12_mag, S12_phase = caculate_S_param(array0, array2)
    S21_mag, S21_phase = caculate_S_param(array1, array3)
    S22_mag, S22_phase = caculate_S_param(array2, array3)
    
    ''' 
    print(f"Calculated S11 magnitude is {S11_mag:.2f}.")
    print(f"Calculated S11 phase is {S11_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
    print(f"Calculated S12 magnitude is {S12_mag:.2f}.")
    print(f"Calculated S12 phase is {S12_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
    print(f"Calculated S21 magnitude is {S21_mag:.2f}.")
    print(f"Calculated S21 phase is {S21_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
    print(f"Calculated S22 magnitude is {S22_mag:.2f}.")
    print(f"Calculated S22 phase is {S22_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
    '''

    return S11_mag, S11_phase, S12_mag, S12_phase, S21_mag, S21_phase, S22_mag, S22_phase

In [16]:
#For one pass of a frequency list 
frequency_list = source.generate_freq_points()

# Initialize empty arrays for S-parameters
plot_S11_mag = np.zeros(len(frequency_list))
plot_S11_phase = np.zeros(len(frequency_list))
plot_S12_mag = np.zeros(len(frequency_list))
plot_S12_phase = np.zeros(len(frequency_list))
plot_S21_mag = np.zeros(len(frequency_list))
plot_S21_phase = np.zeros(len(frequency_list))
plot_S22_mag = np.zeros(len(frequency_list))
plot_S22_phase = np.zeros(len(frequency_list))

#Iterate through frequencies and compute S-parameters
for i in range(1, len(frequency_list)): #skips the first set
    source.set_frequency(frequency_list[i])  # Set the current frequency
    out_buffer = read_dma()  # Read the data buffer
    S11_mag, S11_phase, S12_mag, S12_phase, S21_mag, S21_phase, S22_mag, S22_phase = extract_S_param(out_buffer)
    
    # Store magnitude and phase for each S-parameter
    plot_S11_mag[i] = S11_mag
    plot_S11_phase[i] = S11_phase
    plot_S12_mag[i] = S12_mag
    plot_S12_phase[i] = S12_phase
    plot_S21_mag[i] = S21_mag
    plot_S21_phase[i] = S21_phase
    plot_S22_mag[i] = S22_mag
    plot_S22_phase[i] = S22_phase

Setting frequency to 211919191.91919193
Setting frequency to 413838383.83838385
Setting frequency to 615757575.7575758
Setting frequency to 817676767.6767677
Setting frequency to 1019595959.5959597
Setting frequency to 1221515151.5151515
Setting frequency to 1423434343.4343436
Setting frequency to 1625353535.3535354
Setting frequency to 1827272727.2727273
Setting frequency to 2029191919.1919193
Setting frequency to 2231111111.111111
Setting frequency to 2433030303.030303
Setting frequency to 2634949494.949495
Setting frequency to 2836868686.868687
Setting frequency to 3038787878.787879
Setting frequency to 3240707070.707071
Setting frequency to 3442626262.6262627
Setting frequency to 3644545454.5454545
Setting frequency to 3846464646.464647
Setting frequency to 4048383838.3838387
Setting frequency to 4250303030.3030305
Setting frequency to 4452222222.222222
Setting frequency to 4654141414.141415
Setting frequency to 4856060606.060606
Setting frequency to 5057979797.979798
Setting frequ

In [29]:
#Plot

# Initialize data lists
freq_data = frequency_list #Set up the frequency plot list
sparam_x_axis = [frequency_list, frequency_list, frequency_list, frequency_list]
# Initialize the plot
sparam_fig = plt.figure(title="S-Parameters Plotted", animation_duration=0)
sparam_line = plt.plot([], [], colors=["blue", "red", "green", "orange"])  # Initial empty plot

plt.xlim(0, frequency_list[len(frequency_list)-1])  # Initial X-axis range, will update dynamically
plt.xlabel("Frequency Measured [Hz]")

# Function to update the plot with new sensor data
def update_sparam_plot():
    while True:
        if sparam_is_running.value:
            #Insert function to update s-parameters if needed here
            #This would be the DMA stuff
            
            sparam_line.x = sparam_x_axis
            sparam_line.y = [plot_S11_mag, plot_S12_mag, plot_S21_mag, plot_S22_mag]
            
            
# Toggle button to start/stop the plot
sparam_is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(sparam_is_running, sparam_fig)

# Function to start the thread for continuous plotting
def sparam_start_plot(change):
    if sparam_is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        sparam_thread = Thread(target=update_sparam_plot, daemon=True)
        sparam_thread.start()

# Watch the button and start the plot when pressed
sparam_is_running.observe(sparam_start_plot, names='value')

ToggleButton(value=False, description='Running', icon='play', tooltip='Start/Stop the live plot')

Figure(axes=[Axis(label='Frequency Measured [Hz]', scale=LinearScale(max=20000000000.0, min=0.0)), Axis(orient…

In [22]:
print(plot_S12_mag)

[0.         0.74426364 0.75652466 0.7785265  0.76013971 0.79561748
 0.71971755 0.7777419  0.78008079 0.74491688 0.69356325 0.74662657
 0.71228221 0.77639836 0.77254452 0.70124769 0.83467273 0.83535265
 0.79330249 0.80555861 0.71867019 0.81608922 0.82417681 0.77163109
 0.72267255 0.74933816 0.78710265 0.68787424 0.80994449 0.77439398
 0.76387247 0.75769437 0.81639208 0.76773228 0.69569597 0.67545771
 0.78526264 0.81324886 0.75658319 0.7444876  0.79486045 0.72968413
 0.72291909 0.70456586 0.74700482 0.80610115 0.82753605 0.77252223
 0.77314518 0.7660555  0.80377894 0.79539233 0.86940348 0.77976774
 0.80017644 0.793134   0.7442809  0.72326211 0.76250587 0.75360392
 0.77067166 0.74464433 0.74086128 0.71453795 0.74395344 0.76897237
 0.73455251 0.78410202 0.6981027  0.72566852 0.79399354 0.81954299
 0.79123644 0.80946858 0.81450155 0.83964889 0.84517878 0.84309993
 0.82556639 0.80938438 0.77446677 0.79860329 0.83065199 0.83871985
 0.73347906 0.75587233 0.7938881  0.80243925 0.78297641 0.8386

In [ ]:
# Plot Size
PLOT_SIZE = 3e-6
# Initialize data lists
time_data = np.linspace(0, T, n)  # Time data (X-axis)
x_axis = [time_data, time_data, time_data, time_data]
# Initialize the plot
fig = plt.figure(title="Real-time Sensor Data", animation_duration=0)
line = plt.plot([], [], colors=["blue", "red", "green", "orange"])  # Initial empty plot
plt.xlim(0, PLOT_SIZE)  # Initial X-axis range 
plt.xlabel("Time [s]")

# Function to update the plot with new sensor data
def update_plot():
    while True:
        if is_running.value:
            out_buffer = read_dma()

            # Convert the entire out_buffer into a NumPy array
            out_buffer_np = np.array(out_buffer, dtype=np.int32)  # Assuming `out_buffer` is the raw input array

            # Split into 4 sub-arrays
            out_buffer0 = out_buffer_np[0::4]
            out_buffer1 = out_buffer_np[1::4]
            out_buffer2 = out_buffer_np[2::4]
            out_buffer3 = out_buffer_np[3::4]

            # Process all buffers at once
            real0, imag0 = VNAfunc.iq_break_data_np(out_buffer0)
            real1, imag1 = VNAfunc.iq_break_data_np(out_buffer1)
            real2, imag2 = VNAfunc.iq_break_data_np(out_buffer2)
            real3, imag3 = VNAfunc.iq_break_data_np(out_buffer3)
                
            #Caculate the S-parameters 
            #For caculating the S-parameters combine into complex lists
            array0 = real0 + 1j*imag0 
            array1 = real1 + 1j*imag1
            array2 = real2 + 1j*imag2 
            array3 = real3 + 1j*imag3 

            S11_mag, S11_phase = VNAfunc.caculate_S_param(array0, array1)
            S12_mag, S12_phase = VNAfunc.caculate_S_param(array0, array2)
            S21_mag, S21_phase = VNAfunc.caculate_S_param(array1, array3)
            S22_mag, S22_phase = VNAfunc.caculate_S_param(array2, array3)

#             print(f"Calculated S11 magnitude is {S11_mag:.2f}.")
#             print(f"Calculated S11 phase is {S11_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S12 magnitude is {S12_mag:.2f}.")
#             print(f"Calculated S12 phase is {S12_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S21 magnitude is {S21_mag:.2f}.")
#             print(f"Calculated S21 phase is {S21_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S22 magnitude is {S22_mag:.2f}.")
#             print(f"Calculated S22 phase is {S22_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}."
            
            # Update the plot with new data
            line.x = x_axis
            line.y = [real0, real1, real2, real3]
            
# Toggle button to start/stop the plot
is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(is_running, fig)

# Function to start the thread for continuous plotting
def start_plot(change):
    if is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        thread = Thread(target=update_plot, daemon=True)
        thread.start()

# Watch the button and start the plot when pressed
is_running.observe(start_plot, names='value')